<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/3p_integrations/llamaindex/dlai_agentic_rag/Building_Agentic_RAG_with_Llamaindex_L3_Building_an_Agent_Reasoning_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook ports the DeepLearning.AI short course [Building Agentic RAG with Llamaindex Lesson 3 Building an Agent Reasoning Loop](https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/4/building-an-agent-reasoning-loop) to using Llama 3. 

You should take the course before or after going through this notebook to have a deeper understanding.

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-groq

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Load the data

In [4]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-07-03 16:20:59--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  6.46MB/s    in 2.5s    

2024-07-03 16:21:02 (6.46 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



## Setup the Query Tools

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(
        query: str,
        page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response


    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool

In [7]:
vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Llama and Agent

Note: The LlamaIndex's FunctionCallingAgentWorker API doesn't work correctly with Groq Llama, so we use ReActAgent here.

In [ ]:
import os 

os.environ['GROQ_API_KEY'] = 'xxx' # get a free key at https://console.groq.com/keys

In [ ]:
from llama_index.llms.groq import Groq

from llama_index.core import Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

llm = Groq(model="llama3-70b-8192", temperature=0)
Settings.llm = llm

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [8]:
from llama_index.core.agent import ReActAgent

query_engine_tools = [vector_tool, summary_tool]

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [9]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, and how they communicate with each other."
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'agent roles in MetaGPT', 'page_numbers': None}
Observation: In MetaGPT, there are five defined roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has a specific profile, goal, and constraints, and is initialized with specific context and skills.
Thought: I have some information about the agent roles in MetaGPT, but I need more information about how they communicate with each other.
Action: vector_tool_metagpt
Action Input: {'query': 'communication between agent roles in MetaGPT'}
Observation: In MetaGPT, agents communicate through structured outputs such as documents and diagrams, rather than dialogue. This communication is facilitated by a shared message pool, where agents publish their outputs and can access messages from other entities transparently. Additionally, a subscription mechanis

In [10]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 4
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-07-03
last_modified_date: 2024-07-03

Preprint
Figure 2: An example of the communication protocol (left) and iterative programming with exe-
cutable feedback (right). Left: Agents use a shared message pool to publish structured messages.
They can also subscribe to relevant messages based on their profiles. Right : After generating the
initial code, the Engineer agent runs and checks for errors. If errors occur, the agent checks past
messages stored in memory and compares them with the PRD, system design, and code files.
3 M ETAGPT: A M ETA-PROGRAMMING FRAMEWORK
MetaGPT is a meta-programming framework for LLM-based multi-agent systems. Sec. 3.1 pro-
vides an explanation of role specialization, workflow and structured communication in this frame-
work, and illustrates how to organize a multi-agent system within the context of SOPs. Sec. 3.2
presents a communicati

In [11]:
response = agent.query(
    "Tell me about the evaluation datasets used."
)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation datasets', 'page_numbers': None}
Observation: The evaluation dataset mentioned is the SoftwareDev dataset, which includes 70 diverse software development tasks.
Thought: I have some information about the evaluation datasets, but I want to know more.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation datasets besides SoftwareDev'}
Observation: HumanEval and MBPP.
Thought: I have a good understanding of the evaluation datasets used, including SoftwareDev, HumanEval, and MBPP.
Answer: The evaluation datasets used in the MetaGPT paper include SoftwareDev, which consists of 70 diverse software development tasks, HumanEval, and MBPP.


In [12]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 23
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-07-03
last_modified_date: 2024-07-03

Preprint
Figure 10: The “Drawing App” generated by MetaGPT.
C E XPERIMENTS
C.1 D ETAILS OF THE SOFTWARE DEVDATASET
The SoftwareDev dataset includes 70 diverse software development tasks. Table 8 displays the
names and detailed prompts of 11 tasks within the dataset. Note that the first seven tasks listed are
used in the main experiments of this paper.
C.2 A DDITIONAL RESULTS
Quantitative results of MetaGPT As shown in Table 4, MetaGPT achieves an average score
of 3.9, surpassing ChatDev’s score of 2.1 Zhao et al. (2023), which is based on the Chat chain.
Compare the scores of general intelligent algorithms, including AutoGPT Torantulino et al. (2023),
which all score 1.0, failing to generate executable code. We observe that the generated code is often
short, lacks comprehensive logic, and tends to fail to handle cross-file

In [13]:
# use agent.chat instead of agent.query to pass conversational history automatically to answer follow up questions
response = agent.chat("Tell me the results over one of the above datasets.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'results over softwaredev dataset', 'page_numbers': None}
Observation: The results of MetaGPT over the SoftwareDev dataset include an average score of 3.9, surpassing ChatDev's score of 2.1. Additionally, the results show that MetaGPT achieves a score of 3.75 in terms of executability, which is very close to 4 (flawless). It also takes less time (503 seconds) and requires fewer tokens to generate one line of code compared to ChatDev.
Thought: I have the necessary information to answer the question without using any more tools. I'll use the user's language to answer
Answer: According to the MetaGPT paper, the model achieves an average score of 3.9 on the SoftwareDev dataset, outperforming ChatDev's score of 2.1. It also demonstrates strong executability with a score of 3.75, and is more efficient in terms of time and tokens requi

In [14]:
# use agent.chat instead of agent.query to pass conversational history automatically to answer follow up questions
response = agent.chat("Tell me more.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation results on SoftwareDev dataset', 'page_numbers': None}
Observation: The evaluation results on the SoftwareDev dataset show that MetaGPT achieves an average score of 3.9, surpassing ChatDev's score of 2.1. Additionally, MetaGPT outperforms ChatDev on nearly all metrics, including executability, code statistics, and cost of human revision.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Thought: I need to get more information about the evaluation results on the SoftwareDev dataset.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation results on SoftwareDev dataset in detail'}
Observation: The evaluation results on the SoftwareDev dataset are presented in Table 9, which shows the performance of MetaGPT with GPT-4 32K on 11 tasks within the SoftwareDev data

## Lower-Level: Debuggability and Control

Note: The LlamaIndex's FunctionCallingAgentWorker API doesn't work correctly with Groq Llama, so we use ReActAgent here.

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [15]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [16]:
step_output = agent.run_step(task.task_id)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'agent roles in MetaGPT'}
Observation: In MetaGPT, there are five defined agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has a specific profile, goal, and constraints, and is initialized with specific context and skills.


In [17]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 08e7253c-842a-457e-8eda-9d2822b8540d: 1
In MetaGPT, there are five defined agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has a specific profile, goal, and constraints, and is initialized with specific context and skills.


In [18]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 08e7253c-842a-457e-8eda-9d2822b8540d: 1


TaskStep(task_id='08e7253c-842a-457e-8eda-9d2822b8540d', step_id='76b6833d-6083-4d3e-808e-55ec9e667fc3', input=None, step_state={'is_first': False}, next_steps={}, prev_steps={}, is_ready=True)

In [19]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
Thought: I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'how agents share information in MetaGPT'}
Observation: In MetaGPT, agents share information through a global message pool, where they publish their structured messages and can access messages from other entities transparently. This allows any agent to directly retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses. Additionally, a subscription mechanism is used to filter out irrelevant information, where agents can select information to follow based on their role profiles, ensuring they only receive task-related information and avoid distractions.


In [20]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In MetaGPT, agents share information through a global message pool, where they publish their structured messages and can access messages from other entities transparently. This allows any agent to directly retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses. Additionally, a subscription mechanism is used to filter out irrelevant information, where agents can select information to follow based on their role profiles, ensuring they only receive task-related information and avoid distractions.
True


In [21]:
response = agent.finalize_response(task.task_id)

In [22]:
print(str(response))

In MetaGPT, agents share information through a global message pool, where they publish their structured messages and can access messages from other entities transparently. This allows any agent to directly retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses. Additionally, a subscription mechanism is used to filter out irrelevant information, where agents can select information to follow based on their role profiles, ensuring they only receive task-related information and avoid distractions.
